In [35]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json


In [36]:
requests = pd.read_csv("requests.csv")
requests.head()

,Timestamp,Requested_Pickup,Requested_Dropoff,Actual_Pickup,Actual_Dropoff
0,1380949200,1149,725,218,3601
1,1380949200,1225,1714,2003,3601
2,1380949200,1237,3143,1314,2218
3,1380949200,1323,3566,1330,2573
4,1380949200,1328,2466,1323,2473


In [37]:
requests['Hour'] = requests['Timestamp'].apply(lambda x: pd.Timestamp(x,unit='s',tz='America/New_York').hour)

In [38]:
requests['Serving'] = requests['Actual_Dropoff'].apply(lambda x: 0 if x ==-1 else 1 )

In [39]:
requests.head()

,Timestamp,Requested_Pickup,Requested_Dropoff,Actual_Pickup,Actual_Dropoff,Hour,Serving
0,1380949200,1149,725,218,3601,1,1
1,1380949200,1225,1714,2003,3601,1,1
2,1380949200,1237,3143,1314,2218,1,1
3,1380949200,1323,3566,1330,2573,1,1
4,1380949200,1328,2466,1323,2473,1,1


In [40]:
unserving = requests[requests['Serving']==0]

In [41]:
unserving.head()

,Timestamp,Requested_Pickup,Requested_Dropoff,Actual_Pickup,Actual_Dropoff,Hour,Serving
688,1380949380,1225,1279,-1,-1,1,0
689,1380949380,1260,1999,-1,-1,1,0
692,1380949380,1387,1034,-1,-1,1,0
694,1380949380,1389,2569,-1,-1,1,0
709,1380949380,2013,1998,-1,-1,1,0


In [42]:
pick_up_location_by_time = unserving.groupby(['Hour','Requested_Pickup']).count()['Timestamp'].reset_index()

In [43]:
pick_up_location_by_time.rename(columns={'Timestamp':'Count'}, inplace=True)
pick_up_location_by_time.head()

,Hour,Requested_Pickup,Count
0,0,0,1
1,0,2,5
2,0,3,4
3,0,4,1
4,0,6,7


In [44]:
geomanhattan = gpd.read_file('manhattan.geojson')
geomanhattan.count()

id           4411
w            9625
geometry    14036
dtype: int64

In [45]:
geomanhattan.dropna(subset=['id'], inplace=True)

In [46]:
allpoint= pd.DataFrame({'id' : np.tile(list(range(4411)), 24),'hour':np.tile(list(range(24)),4411)})

In [47]:
pick_up_location_by_time_all = pd.merge(allpoint,pick_up_location_by_time, 
                                        how='left',left_on=['id','hour'],right_on=['Requested_Pickup','Hour'])

In [52]:
pick_up_location_by_time_all = pick_up_location_by_time_all[['id','hour','Count']].fillna(0)
pick_up_location_by_time_all.head()

,id,hour,Count
0,0,0,1.0
1,1,1,0.0
2,2,2,0.0
3,3,3,0.0
4,4,4,0.0


In [10]:
#pick_up_location_by_time.to_json(r'pick_up_location_by_time.json',orient='split')

In [11]:
#pick_up_location_by_time.to_csv(r'pick_up_location_by_time.csv')


In [56]:
servingrate = pd.read_csv("servingrate.csv")
servingrate.head()

,Unnamed: 0,Hour,Total,Serving,rate
0,0,0,19313,6748,0.349402
1,1,1,10052,7297,0.725925
2,2,2,6070,5464,0.900165
3,3,3,3725,3443,0.924295
4,4,4,2567,2310,0.899883


In [58]:
pick_up_location_by_time = pick_up_location_by_time_all.merge(servingrate, left_on='hour',right_on='Hour')
pick_up_location_by_time.head()

,id,hour,Count,Unnamed: 0,Hour,Total,Serving,rate
0,0,0,1.0,0,0,19313,6748,0.349402
1,24,0,49.0,0,0,19313,6748,0.349402
2,48,0,0.0,0,0,19313,6748,0.349402
3,72,0,0.0,0,0,19313,6748,0.349402
4,96,0,0.0,0,0,19313,6748,0.349402


In [60]:
records = []
for key, grp in pick_up_location_by_time.groupby(['Hour','rate']):
    records.append({
        "hour": int(key[0]),
        "rate": float(key[1]),
        "perID": {
            row.id: row.Count for row in grp.itertuples()
        }})

In [61]:
records

[{'hour': 0,
  'rate': 0.34940195723088074,
  'perID': {0: 1.0,
   24: 49.0,
   48: 0.0,
   72: 0.0,
   96: 0.0,
   120: 0.0,
   144: 0.0,
   168: 0.0,
   192: 1.0,
   216: 11.0,
   240: 4.0,
   264: 0.0,
   288: 0.0,
   312: 0.0,
   336: 5.0,
   360: 5.0,
   384: 0.0,
   408: 4.0,
   432: 5.0,
   456: 0.0,
   480: 1.0,
   504: 6.0,
   528: 0.0,
   552: 0.0,
   576: 3.0,
   600: 5.0,
   624: 7.0,
   648: 1.0,
   672: 0.0,
   696: 1.0,
   720: 13.0,
   744: 1.0,
   768: 0.0,
   792: 0.0,
   816: 0.0,
   840: 0.0,
   864: 2.0,
   888: 0.0,
   912: 0.0,
   936: 0.0,
   960: 1.0,
   984: 0.0,
   1008: 0.0,
   1032: 6.0,
   1056: 3.0,
   1080: 20.0,
   1104: 10.0,
   1128: 8.0,
   1152: 0.0,
   1176: 1.0,
   1200: 4.0,
   1224: 16.0,
   1248: 7.0,
   1272: 2.0,
   1296: 0.0,
   1320: 2.0,
   1344: 0.0,
   1368: 0.0,
   1392: 11.0,
   1416: 0.0,
   1440: 0.0,
   1464: 2.0,
   1488: 0.0,
   1512: 12.0,
   1536: 0.0,
   1560: 0.0,
   1584: 2.0,
   1608: 0.0,
   1632: 7.0,
   1656: 1.0,
   1680

In [62]:
with open('pick_up_location_by_time.json', 'w',encoding='utf-8') as f:
     json.dump(records, f, ensure_ascii=False, indent=4)

# Aggregate to Zipcode 

In [65]:
geomanhattan = gpd.read_file('manhattan.geojson')
geozip = gpd.read_file('nyc_zip.geojson')

In [66]:
geomanhattan.dropna(subset=['id'], inplace=True)

In [67]:
zipid = gpd.sjoin(geomanhattan,geozip)

In [68]:
zipid.head()

,id,w,geometry,index_right,zipcode
0,0.0,NaN,POINT (-73.9975944 40.7140611),187,10013
1,1.0,NaN,POINT (-73.9980743 40.7137811),187,10013
28,28.0,NaN,POINT (-74.00596609999999 40.7203782),187,10013
29,29.0,NaN,POINT (-74.00641950000001 40.7204244),187,10013
30,30.0,NaN,POINT (-74.006055 40.7244231),187,10013


In [80]:
zipcode_id = zipid[['id','zipcode','geometry']]
zipcode_id.head()

,id,zipcode,geometry
0,0.0,10013,POINT (-73.9975944 40.7140611)
1,1.0,10013,POINT (-73.9980743 40.7137811)
28,28.0,10013,POINT (-74.00596609999999 40.7203782)
29,29.0,10013,POINT (-74.00641950000001 40.7204244)
30,30.0,10013,POINT (-74.006055 40.7244231)


In [83]:
unserving = unserving.merge(zipcode_id,left_on='Requested_Pickup',right_on='id')

In [85]:
pick_up_location_by_time_zipcode = unserving.groupby(['Hour','zipcode']).count()['id'].reset_index()

In [97]:
pick_up_location_by_time_zipcode.rename(columns={'id':'Count'}, inplace=True)
pick_up_location_by_time_zipcode.head()


,Hour,zipcode,Count
0,0,00083,85
1,0,10001,1280
2,0,10002,430
3,0,10003,1422
4,0,10004,21


In [91]:
nyc_zip = list(geozip['zipcode'])

In [96]:
allpoint2= pd.DataFrame({'zipcode' : np.tile(nyc_zip, 24),'hour':np.tile(list(range(24)),263)})

In [98]:
pick_up_location_by_time_zipcode_all = pd.merge(allpoint2,pick_up_location_by_time_zipcode, 
                                        how='left',left_on=['zipcode','hour'],right_on=['zipcode','Hour'])

In [100]:
pick_up_location_by_time_zipcode_all = pick_up_location_by_time_zipcode_all[['zipcode','hour','Count']].fillna(0)
pick_up_location_by_time_zipcode_all.head()

,zipcode,hour,Count
0,10471,0,0.0
1,10463,1,0.0
2,10475,2,0.0
3,10464,3,0.0
4,11222,4,0.0


In [103]:
pick_up_location_by_time_zipcode = pick_up_location_by_time_zipcode_all.merge(servingrate, left_on='hour',right_on='Hour')
pick_up_location_by_time_zipcode.head()

,zipcode,hour,Count,Unnamed: 0,Hour,Total,Serving,rate
0,10471,0,0.0,0,0,19313,6748,0.349402
1,11101,0,0.0,0,0,19313,6748,0.349402
2,11105,0,0.0,0,0,19313,6748,0.349402
3,11691,0,0.0,0,0,19313,6748,0.349402
4,10279,0,0.0,0,0,19313,6748,0.349402


In [104]:
records = []
for key, grp in pick_up_location_by_time_zipcode.groupby(['Hour','rate']):
    records.append({
        "hour": int(key[0]),
        "rate": float(key[1]),
        "zipcode": {
            row.zipcode: row.Count for row in grp.itertuples()
        }})

In [105]:
with open('pick_up_location_by_time_zipcode.json', 'w',encoding='utf-8') as f:
     json.dump(records, f, ensure_ascii=False, indent=4)